## GLMAT: 2D kernel

In [ ]:
from __future__ import print_function, division
import os
import sys
import numpy as np
import scipy as sp
from numba import jit, guvectorize

from dstrf import mat

# plotting packages
%matplotlib inline
import matplotlib.pyplot as plt # plotting functions
import seaborn as sns           # data visualization package
sns.set_style("whitegrid")


In [ ]:
# model parameters: (α1, α2, ω, τ1, τ2, tref)
matparams = np.asarray([100, 2, 7, 10, 200, 2], dtype='d')
model_dt = 0.5

# data parameters
duration = 40000
n_samples = int(duration / model_dt)
n_assim = 3
n_test = 0

# assimilation parameters
nthreads = 10
nwalkers = 2000
nsteps = 500
matparams_i = [0,1,2]
matparams_n = len(matparams_i)
matparams_f = matparams[matparams_i]

In [ ]:
# STRF: keep this very simple for proof of principle
stim_dt = 10.0
nfreq = 30
ntau  = 30

from scipy.signal import resample
filts = np.load('../../filters.npz')
print(filts.keys())
k1 = resample(filts['bbs'], nfreq, axis=0)[:,:ntau] * 10

def filter_stimulus(S, kernel):
    """Convolve spectrogram S with spectrotemporal kernel. Kernel should not be flipped."""
    nf, nt = S.shape
    X = np.zeros(nt)
    for i in range(nf):
        X += np.convolve(S[i], kernel[i], mode="full")[:nt]
    return X

def fft_filter_stimulus(Sf, kernel):
    """Convolution of S with kernel. Sf is np.fft.fft(S, n=(S.size + kernel.size))"""
    nfft = Sf.shape[1]
    nt = nfft - kernel.shape[1] + 1
    K = np.fft.fft(kernel, n=nfft, axis=1)
    X = np.fft.ifft(Sf * K, axis=1).real[:, :nt]
    return X.sum(0)
        
def upsample_convolution(X, upsample):
    t = np.arange(X.size)
    tu = np.arange(0, X.size, 1 / upsample)
    return np.interp(tu, t, X)

sns.heatmap(k1)


In [ ]:
# generate some random data to fit
np.random.seed(1)
stim_dt = 10.0
upsample = int(stim_dt / model_dt)
stim = np.random.randn(nfreq, int(n_samples / (stim_dt / model_dt)))
stim[:,:100] = 0
#stim_mat = strf.design_matrix(stim, ntau)
#stim_fft = np.fft.fft(stim, n=(stim.shape[1] + ntau - 1))
plt.imshow(stim, aspect='auto')

In [ ]:
@jit
def predict_spikes(V, params, dt):
    N = V.size
    S = np.zeros(N, dtype='i')
    P = np.zeros(N, dtype='d')
    R = np.random.uniform(size=N)
    omega = params[2]
    A1 = np.exp(-dt / params[3])
    A2 = np.exp(-dt / params[4])
    i_refrac = int(params[5] / dt)
    H1 = H2 = 0
    iref = 0
    for i in range(N):
        Vt = V[i]
        H1 *= A1
        H2 *= A2
        p = np.exp(Vt - H1 - H2 - omega) * dt
        if i > iref and p > R[i]:
            H1 += params[0]
            H2 += params[1]
            iref = i + i_refrac
            S[i] = 1
    return S

@jit(nopython=True)
def lci_poisson(V, H, spikes, params, dt):
    mu = V - H[:, 0] * params[0] - H[:, 1] * params[1] - params[2]
    lp = spikes * mu - dt * np.exp(mu)
    return lp.sum()

In [ ]:
data = []
V = upsample_convolution(filter_stimulus(stim, k1), upsample)
#V = upsample_convolution(fft_filter_stimulus(stim_fft, k1), upsample)
for i in range(n_assim + n_test):
    spikes = predict_spikes(V, matparams, model_dt)
    H = np.c_[mat.predict_adaptation(spikes, matparams[3], model_dt),
              mat.predict_adaptation(spikes, matparams[4], model_dt)]
    z = np.nonzero(spikes)[0]
    d = {"V": V,
         "H": H,
         "duration": duration,
         "spike_t": z, 
         "spike_v": spikes,
         "lci": lci_poisson(V, H, spikes, matparams, model_dt)
         }
    data.append(d)

# split into assimilation and test sets
assim_data = data[:n_assim]
test_data = data[n_assim:]

In [ ]:
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)
ax1.plot(V)
ax1.plot(H)
for i, d in enumerate(data):
    ax2.vlines(d["spike_t"], i, i + 0.5)
ax1.set_xlim(0, 8000)
ax2.set_xlim(0, 8000)
print(sum(d["lci"] for d in data))
len(data[0]["spike_t"])

In [ ]:
# estimate parameters using emcee
from neurofit import priors, costs, utils, startpos
import emcee

mat_prior = priors.joint_independent(
                [ priors.uniform(-50,  200),
                  priors.uniform(-5,   10),
                  priors.uniform( 0,  20),
                ])
startparams = np.asarray([[-50, 200],
                          [-5, 10],
                          [0, 20],
                         ], dtype='d')

# lasso prior on RF parameters
rf_lambda = 1.0

# this is the local copy of the parameters that we'll update in each step
mparams = matparams.copy()
rfparams = k1.flatten()

@jit
def lnpost_dyn(theta):
    """Posterior probability for dynamical parameters"""
    mparams[matparams_i] = theta[:matparams_n]
    rfparams[:] = theta[matparams_n:]
    rf_prior = -np.sum(np.abs(theta)) * rf_lambda
    ll = mat_prior(theta[:matparams_n]) + rf_prior
    if not np.isfinite(ll):
        return -np.inf
    lp = 0
    V = upsample_convolution(filter_stimulus(stim, rfparams.reshape(nfreq, ntau)), upsample)
    for d in assim_data:
        lp += lci_poisson(V, d["H"], d["spike_v"], mparams, model_dt)
    return ll + lp        

In [ ]:
# theoretically this is as good as it can get
theta_true = np.concatenate([matparams_f, rfparams])
print("lnpost of p_true: {}".format(lnpost_dyn(theta_true)))
# and this is our initial state
p0 = np.concatenate([startpos.uniform_independent(nwalkers, startparams[:,0], startparams[:,1]),
                     startpos.normal_independent(nwalkers, rfparams, [0.1] * rfparams.size)],
                   axis=1)
theta_0 = np.median(p0, 0)
print("lnpost of p0 median: {}".format(lnpost_dyn(theta_0)))
%timeit lnpost_dyn(theta_true)
%time for theta_0 in p0: lnpost_dyn(theta_0)

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, theta_true.size, lnpost_dyn, threads=nthreads)
tracker = utils.convergence_tracker(nsteps, 25)

for pos, prob, _ in tracker(sampler.sample(p0, iterations=nsteps, storechain=False)): 
    continue

In [ ]:
print("lnpost of p median: {}".format(np.median(prob)))
print("average acceptance fraction: {}".format(sampler.acceptance_fraction.mean()))
theta = np.median(pos, 0)
mparams[matparams_i] = theta[:matparams_n]
rfparams[:] = theta[matparams_n:]
k1_est = rfparams.reshape(nfreq, ntau)
plt.subplot(121)
sns.heatmap(k1)
plt.subplot(122)
sns.heatmap(k1_est)
print(matparams_f)
print(theta[:matparams_n])

In [ ]:
from corner import corner
sns.set_style("whitegrid")

mpos = pos[:,:matparams_n]
matlabs = ['a1','a2','b','w', 'R']
c = corner(mpos,
           range=[sp for sp in startparams],
       bins=50, smooth=2,smooth1d=0,
       labels=matlabs,
       truths=matparams_f)

In [ ]:
# see how well predictions line up
d = assim_data[0]
Vpred = upsample_convolution(filter_stimulus(stim, k1_est), upsample)

#Y, S = matmodel.predict(matstate, mparams, I, model_dt, upsample=upsample, stochastic=True)
plt.plot(d["V"][:4000])
plt.plot(Vpred[:4000])

In [ ]:
for i, d in enumerate(data):
    plt.vlines(d["spike_t"], i, i + 0.5, 'r')

for i in range(len(data), len(data) + 10):
    S = predict_spikes(Vpred, mparams, model_dt)
    spk_t = S.nonzero()[0]
    plt.vlines(spk_t, i, i + 0.5)

plt.xlim(0, 10000)